# Challenge 04-B - Retrieval Augmented Generation (RAG) for Unstructured Data


## Introduction

Businesses have a lot of proprietary information that needs to be taken into account when answering user's questions - these cannot always be answered through the data that the GPT models have been trained on. 

In the last notebook, we worked with structured data primarily. A lot of the time, your enterprise data is not limited to just structured formats like CSV files or SQL tables. It may also include unstructured data like PDF documents or images. In fact, your individual documents could have both unstructured and structured data built into them. Extracting information from these diverse formats in a comprehensible manner presents a challenge. Tools like Azure Form Recognizer enable the extraction of data from unstructured sources such as forms or documents. Once the data is extracted into a structured JSON format, then Cognitive Search can be utilized to consolidate the entire information from different data types into indexes, facilitating the retrieval of relevant documents.

In this notebook, we will walk you through a use case of Retrieval Augmented Generation (RAG) that involves working with unstructured data. The RAG approach combines various technologies to enhance the quality and relevance of generated outputs. We will leverage Azure Form Recognizer to process complex documents, utilizing the layout API to extract text and tables effectively. We will utilize Azure Cognitive Search to create an index by configuring semantic search capabilities, enabling the retrieval of relevant document pages. Additionally, embeddings will be incorporated to retrieve content that is more closely aligned with the user's question. Finally, Azure OpenAI's ChatGPT model will utilize the extracted content to generate a more meaningful answer. It is important to emphasize that this grounding process follows the RAG pattern mentioned in the previous notebook and helps eliminate inaccuracies in the generated responses.

Your goals for this challenge are to read through this notebook, run each code block, observe the results, and then be able to answer the questions posed in the student guide.


In [1]:
! pip install "tiktoken==0.4.0" 

In [1]:
# Import Azure Forms Recognizer, Azure Cognitive Search, OpenAI, and other python modules

import os, json, requests, sys, re
import requests
from pprint import pprint
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings
)

from azure.ai.formrecognizer import DocumentAnalysisClient
import openai
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'azure'

In [3]:
! pip install "azure-ai-formrecognizer==3.3.3"

In [5]:
# This is secure and recommended way to load OpenAI resource credentials and deployment names

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['CHAT_MODEL_NAME']
embedding_model=os.environ['EMBEDDING_MODEL_NAME']

**NOTE:** The path in the code cell below is referring to the `/data/unstructured/raw` folder. You may need to update this path if you are running this notebook from a different location then from where you extracted it.

In [6]:
# -- raw data
RAW_DATA_FOLDER= '../data/unstructured/raw'
# -- extracted json file 
EXTRACTED_DATA_FOLDER = '../data/unstructured/extracted'

In [7]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

We want to extract the data from our unstructured data into a more readable format for the model to understand. The Form Recognizer tool helps us do so by leveraging the prebuilt layout models. Here, we primarily are working with PDFs but we could also have JPG and PNG formats that the form recognizer tool also supports.

For each document, we want to specify the way information is being extracted. For example in this use case, each document has many pages. To keep track of the pages, we store them in page_number. We also want to extract the content for each page and drop it in a page_context field.

In [8]:
def extract_local_single_file(file_name: str):
    not_completed = True
    while not_completed:
        with open(file_name, "rb") as f:
            poller = document_analysis_client.begin_analyze_document(
                "prebuilt-layout", document=f
            )
            not_completed=False
    result = poller.result()
    return get_page_content(file_name, result)

def extract_files( folder_name: str, destination_folder_name: str):
    os.makedirs(destination_folder_name, exist_ok=True)
    for file in os.listdir(folder_name):
        if file[-3:].upper() in ['PDF','JPG','PNG']:
            print('Processing file:', file, end='')
        
            page_content = extract_local_single_file(os.path.join(folder_name, file))
            output_file = os.path.join(destination_folder_name, file[:-3] +'json')
            print(f'  write output to {output_file}')
            with open(output_file, "w") as f:
                f.write(json.dumps(page_content))


def get_page_content(file_name:str, result):
    page_content = []
    for page in result.pages:
        all_lines_content = []
        for line_idx, line in enumerate(page.lines):
            all_lines_content.append(' '.join([word.content for word in line.get_words()]))
        page_content.append({'page_number':page.page_number, 
                                'page_content':' '.join(all_lines_content)})
    return {'filename':file_name, 'content':page_content}





In [11]:
extract_files(RAW_DATA_FOLDER, EXTRACTED_DATA_FOLDER)

Processing file: Self-Consistency_Improves_Chain-of-Thought_Reasonsing_in_LLMs.pdf

HttpResponseError: (InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Inner error: {
    "code": "InvalidContentLength",
    "message": "The input image is too large. Refer to documentation for the maximum file size."
}

## More About our data

For this walkthrough, we will take a look at various Research Papers on LLM topics in PDF documents. This includes topics like autoprompting, chain of thought prompting, precise zero shot dense retrival, and more. This dataset contains various unstructured formats such as text, tables, graphs, and formulas.

## Data Description

The relevant schema for our work today consists of 

- document_id
- document_name
- file_path
- page_number
- page_text


In [26]:
documents=[]
for file in os.listdir(EXTRACTED_DATA_FOLDER):
    with open(os.path.join(EXTRACTED_DATA_FOLDER, file)) as f:
        page_content= json.loads(f.read())
    documents.extend(
        [
            {
                'document_id':page_content['filename'].split('/')[-1].split('.')[0] + '-' + str(page['page_number']),
                'document_name':page_content['filename'].split('/')[-1],
                'file_path':page_content['filename'],              
                'page_number':page['page_number'],
                'page_text':page['page_content']
            }
            for page in page_content['content']
        ]
    )

In [27]:
len(documents)

14

In [21]:
#Example of a single page of research paper file that will be indexed in Azure Cognitive Search
documents[10]

{'document_id': 'Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning-1',
 'document_name': 'Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning.pdf',
 'file_path': '../data/unstructured/raw/Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning.pdf',
 'page_number': 1,
 'page_text': 'The Power of Scale for Parameter-Efficient Prompt Tuning Brian Lester* Rami Al-Rfou Noah Constant Google Research {brianlester, rmyeid, nconstant } @google. com Abstract In this work, we explore "prompt tuning," a simple yet effective mechanism for learn- ing "soft prompts" to condition frozen lan- guage models to perform specific downstream tasks. Unlike the discrete text prompts used by GPT-3, soft prompts are learned through back- propagation and can be tuned to incorporate signals from any number of labeled examples. Our end-to-end learned approach outperforms GPT-3\'s few-shot learning by a large margin. More remarkably, through ablations on model size using T5, we show that prompt tuning be- comes mo

This section will focus on Cognitive Search and the following topics:
1. Creating an index client
2. Defining the index fields with necessary attributes
3. Creating a semantic configuration
4. Loading our index with the document pages

In [22]:
# Create an SDK client
service_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")   
key = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")
credential = AzureKeyCredential(key)

index_name = "research-paper-index"

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
index_client

In [23]:
fields = [
    SimpleField(name="document_id", type=SearchFieldDataType.String, key=True),
    SimpleField(name="page_number", type=SearchFieldDataType.Int64),
    SimpleField(name="file_path", type=SearchFieldDataType.String),
    SearchableField(name="document_name", type=SearchFieldDataType.String,
                searchable=True, retrievable=True),
    SearchableField(name="page_text", type=SearchFieldDataType.String,
                filterable=True, searchable=True, retrievable=True),
]

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="document_id"),
        prioritized_keywords_fields=[SemanticField(field_name="document_name")],
        prioritized_content_fields=[SemanticField(field_name="page_text")]
    )
)


# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 research-paper-index created


In [24]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 14 documents


In [25]:
len(result)

14

Here we see Azure Cognitive Search in action! We can retrive the most relevant documents out of all the ones that we are working with. 

In [28]:
query = "What is automated prompt engineering?"
count = 10  #Numero de resultados a retornar
results = search_client.search(search_text=query, top=count, include_total_count=True)
page_chunks = []
citations = []
for result in results:
    page_chunks.append(result['page_text'])
    citations.append(result['document_name'])
    
    

In [29]:
embed_df = pd.DataFrame(page_chunks, columns = ["page_chunks"]) #datframe with document chunks
embed_df

,page_chunks
0,Instruction Model Input I instructed my friend...
1,AUTOPROMPT: Eliciting Knowledge from Language ...
2,LARGE LANGUAGE PROMPT ENGINEERS MODELS ARE HUM...
3,Original Input xinp a real joy. AUTOPROMPT Xpr...
4,Generated Knowledge Prompting for Commonsense ...
5,Dataset Question / Knowledge Prediction Score ...
6,The Power of Scale for Parameter-Efficient Pro...
7,Fine-tuning Prefix-Tuning: Optimizing Continuo...
8,Model Tuning Pre-trained Model (11B params) Pr...
9,1 Introduction The NLP landscape has recently ...


Once we have the most relevant documents, let us create embeddings for all the page chunks. This will help us find the most similar documents to our given user query.

In [3]:
# Handling Rate Limits

from openai.error import RateLimitError
from time import sleep


def get_embedding(text: str, engine: str = "text-embedding-ada-002"):
    count=0
    while True:
        try:
            embedding = openai.Embedding().create(input=[text], engine=engine)["data"][0]["embedding"]
            break;
        except RateLimitError:
            count+=1
            #print(f'RateLimitError Count: {count}')
            sleep(2)            
    return np.array(embedding).astype(np.float32)

def get_completion(prompt, model="gpt-35-turbo"): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [31]:
#Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
embed_df['embedding'] = embed_df["page_chunks"].apply(lambda page_text : get_embedding(page_text, engine = embedding_model))

In [32]:
embed_df

,page_chunks,embedding
0,Instruction Model Input I instructed my friend...,"[-0.028272625, 0.002067556, 0.014960588, -0.00..."
1,AUTOPROMPT: Eliciting Knowledge from Language ...,"[-0.020375008, 0.0042142053, 0.00047743105, -0..."
2,LARGE LANGUAGE PROMPT ENGINEERS MODELS ARE HUM...,"[-0.019849496, -0.003594275, 0.00012190554, -0..."
3,Original Input xinp a real joy. AUTOPROMPT Xpr...,"[-0.027736554, -0.014863948, 0.004185375, -0.0..."
4,Generated Knowledge Prompting for Commonsense ...,"[-0.000119818484, 0.015018977, 0.010514665, -0..."
5,Dataset Question / Knowledge Prediction Score ...,"[0.0019514625, 0.0028727974, 0.020901727, -0.0..."
6,The Power of Scale for Parameter-Efficient Pro...,"[-0.017624808, -0.0056814984, 0.025339136, -0...."
7,Fine-tuning Prefix-Tuning: Optimizing Continuo...,"[-0.019425135, 0.009993075, 0.0051297783, -0.0..."
8,Model Tuning Pre-trained Model (11B params) Pr...,"[-0.022517242, -0.008944541, 0.013544987, 0.01..."
9,1 Introduction The NLP landscape has recently ...,"[-0.0014194939, 0.0081906235, 0.014798584, -0...."


In [33]:
query_embedding = get_embedding(query, engine=embedding_model)
embed_df["similarities"] = embed_df['embedding'].apply(lambda page_embedding: cosine_similarity(page_embedding, query_embedding))

top_results = (
    embed_df.sort_values("similarities", ascending=False)
    .reset_index(drop=True)
    .head(3)
)
top_results

,page_chunks,embedding,similarities
0,LARGE LANGUAGE PROMPT ENGINEERS MODELS ARE HUM...,"[-0.019849496, -0.003594275, 0.00012190554, -0...",0.865939
1,AUTOPROMPT: Eliciting Knowledge from Language ...,"[-0.020375008, 0.0042142053, 0.00047743105, -0...",0.841530
2,Original Input xinp a real joy. AUTOPROMPT Xpr...,"[-0.027736554, -0.014863948, 0.004185375, -0.0...",0.838593


In [34]:
prompt = f"""
Provided below are user query and list of extracted pages from research papers separated by triple backticks.
Your task is to extract key pieces of information from that list based on the user query and phrase that as a comprehensive answer. 

User Query: ```{query}```
List of Extracted Pages: ```{top_results['page_chunks'].to_list()}```

Answer:
"""

print(prompt)


Provided below are user query and list of extracted pages from research papers separated by triple backticks.
Your task is to extract key pieces of information from that list based on the user query and phrase that as a comprehensive answer. 

User Query: ```What is automated prompt engineering?```
List of Extracted Pages: ```['LARGE LANGUAGE PROMPT ENGINEERS MODELS ARE HUMAN-LEVEL Yongchao Zhou*,1,2 Andrei Ioan Muresanu*,2,3 Ziwen Han*,1,2 Keiran Paster1,2 Silviu Pitis1,2 Harris Chan1,2 Jimmy Ba1,2 1 University of Toronto 2 Vector Institute 3 University of Waterloo ABSTRACT By conditioning on natural language instructions, large language models (LLMs) have displayed impressive capabilities as general-purpose computers. However, task performance depends significantly on the quality of the prompt used to steer the model, and most effective prompts have been handcrafted by humans. Inspired by classical program synthesis and the human approach to prompt engineering, we propose Automatic 

In [35]:
response = get_completion(prompt, chat_model)
print(response)

Automated prompt engineering is a method for generating and selecting natural language instructions to steer large language models (LLMs) towards desired behaviors. It is inspired by classical program synthesis and the human approach to prompt engineering. The method involves treating the instruction as the "program," optimized by searching over a pool of instruction candidates proposed by an LLM in order to maximize a chosen score function. The quality of the selected instruction is evaluated by the zero-shot performance of another LLM following the selected instruction. Experiments on 24 NLP tasks show that automatically generated instructions outperform the prior LLM baseline by a large margin and achieve better or comparable performance to the instructions generated by human annotators on 19/24 tasks. The method is proposed as a solution to reduce the human effort involved in creating and validating effective instructions.


In [2]:

def query_search(query, count=10):
    results = search_client.search(search_text=query, top=count, include_total_count=True)
    page_chunks = []
    for result in results:
        page_chunks.append(result['page_text'])
        
    #Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
    embed_df['embedding'] = embed_df["page_chunks"].apply(lambda page_text : get_embedding(page_text, engine = embedding_model))

    query_embedding = get_embedding(query, engine=embedding_model)
    embed_df["similarities"] = embed_df['embedding'].apply(lambda page_embedding: cosine_similarity(page_embedding, query_embedding))

    top_results = (
        embed_df.sort_values("similarities", ascending=False)
        .reset_index(drop=True)
        .head(3)
    )
    
    prompt = f"""
    Provided below are user query and list of extracted pages from research papers separated by triple backticks.
    Your task is to extract key pieces of information from that list based on the user query and phrase that as a comprehensive answer. 

    User Query: ```{query}```
    List of Extracted Pages: ```{top_results['page_chunks'].to_list()}```

    Answer:
    """
    
    response = get_completion(prompt, chat_model)
    return response

In [4]:
answer = query_search("How does automated prompt engineering work?", 5)
print(answer)

NameError: name 'search_client' is not defined

In [43]:
answer = query_search("what is prompt tuning?", 10)
print(answer)

Prompt tuning is a mechanism for learning "soft prompts" to condition frozen language models to perform specific downstream tasks. Unlike the discrete text prompts used by GPT-3, soft prompts are learned through backpropagation and can be tuned to incorporate signals from any number of labeled examples. Prompt tuning becomes more competitive with scale and matches the strong performance of model tuning (where all model weights are tuned) as models exceed billions of parameters. This finding is especially relevant because large models are costly to share and serve and the ability to reuse one frozen model for multiple downstream tasks can ease this burden. Prompt tuning only requires storing a small task-specific prompt for each task, and enables mixed-task inference using the original pre-trained model.
